In [5]:
import tweepy
from twarc import Twarc
import pandas as pd
import json
import math
import datetime
from datetime import datetime, timedelta

with open('creds.json') as json_file:
    creds = json.load(json_file)

## Example tweets and retweets according to https://developer.twitter.com/en/docs/twitter-api/v1/tweets/timelines/api-reference/get-statuses-user_timeline

In [6]:
twarc_object = Twarc(creds["client_key"], 
                     creds["client_secret"], 
                     creds["access_token"], 
                     creds["access_token_secret"])

In [7]:
user = "@DEFENSORIAEC"
user = user.replace("@","")
user

'DEFENSORIAEC'

In [13]:


def get_tweets(user, path_last_id='newest_id.json'):
    try:
        
        with open(path_last_id) as json_file:
            data = json.load(json_file)
            if user in data:
                _since_id = data[user]
            else:
                _since_id = None
            
    except FileNotFoundError:
        _since_id = None
        
    
    tweets = []
    retweets = []
    newest_tweet_id = {}

    min_id = math.inf
    max_id = - math.inf

    for tweet in twarc_object.timeline(screen_name = user,
                                       since_id = _since_id,
                                       max_pages = 3):
        access_time = datetime.now().strftime("%Y %b %d %H:%M:%S")                                
        date = datetime.strptime(tweet["created_at"], '%a %b %d %H:%M:%S %z %Y')
        date_5 = date - timedelta(hours = 5)            
        date_format = date_5.strftime("%Y %b %d %H:%M:%S")
        
        # date_tw_format = "%a %b %d %H:%M:%S %Y"
        #print(access_time)
        current_id = tweet["id"]
        if current_id > max_id:
            max_id = current_id
        if current_id < min_id:
            min_id = current_id

        # getting tweets
        if ('RT @' not in tweet["full_text"]):
            
            temp = [tweet["id"], access_time, date_format, user, None, False, None, tweet["full_text"]]
            tweets.append(temp)

        # getting retweets
        else:
           
            temp = [tweet["id"], access_time, date_format, user, None, True,
                    tweet["retweeted_status"]["user"]["screen_name"], tweet["retweeted_status"]["full_text"]]
            retweets.append(temp)
    
    print("Tweets and retweets collected")
            
    if ((max_id != -math.inf) and (max_idx is not None)):
        
        newest_tweet_id[user] = max_id
        
        with open(path_last_id, 'w') as outfile:
            json.dump(newest_tweet_id, outfile)
            
    return tweets, retweets
    

tw, rtw = get_tweets(user)
len(tw), len(rtw)
#print(tweets[1])
     
#tweets#["full_text"]
#retweets[0]#["retweeted_status"]["full_text"]
#min_id, max_id

Tweets and retweets collected


(0, 0)

In [9]:
#tw[0]

#texts = []
#for idx, tweet in enumerate(tweets):
#    texts.append(tweet["full_text"])

In [10]:
#pd.DataFrame(texts)
df_tw = pd.DataFrame(tw, columns=['id', 'fecha_consulta', 'fecha_escritura', 'cuenta_origen', 'query_busqueda',
                                   'retweet', 'retweeted_from', 'texto'])


df_rtw = pd.DataFrame(rtw, columns=['id', 'fecha_consulta', 'fecha_escritura', 'cuenta_origen', 'query_busqueda',
                                   'retweet', 'retweeted_from', 'texto'])

df_rtw.head()

dfs = [df_tw, df_rtw]

t_rt_df = pd.concat(dfs, ignore_index=True)

print(t_rt_df.head(20))


Empty DataFrame
Columns: [id, fecha_consulta, fecha_escritura, cuenta_origen, query_busqueda, retweet, retweeted_from, texto]
Index: []


In [12]:
print(len(t_rt_df))
t_rt_df.to_csv('tw_rtw.csv', sep = '\t', index = True, header=True)

0


## Example getting mentions according to https://developer.twitter.com/en/docs/twitter-api/v1/tweets/timelines/api-reference/get-statuses-mentions_timeline


para este twarc no sirve, entonces mejor usar tweepy search usando los operadoers https://developer.twitter.com/en/docs/twitter-api/v1/rules-and-filtering/search-operators

https://docs.tweepy.org/en/latest/api.html
https://docs.tweepy.org/en/latest/api.html#search-methods


toca verificar que esta opcion no esté incluyendo los tweeets que 

In [38]:
f'to:{user} OR @{user}'

'to:DEFENSORIAEC OR @DEFENSORIAEC'

In [4]:
import tweepy
            
def get_tweets_mentions(user, petitions = 50):
    #add numbers petitions per api -50
    
    try:
        
        with open('newest_id_mentions.json') as json_file:
            data = json.load(json_file)
            if user in data:
                _since_id = data[user]
            else:
                _since_id = None
            
    except FileNotFoundError:
        _since_id = None
    
    _max_id = None
    
    
    max_queries = 430
    min_calls_per_user = petitions
    max_queries_per_user = 120 
    n_tweets_user = max_queries_per_user * 100
    n_queries = 0
    n_queries_user = 0
    
    
    auth = tweepy.AppAuthHandler(creds["client_key"], creds["client_secret"])
    api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

    tweets = tweet_batch = api.search(q = f'to:{user} OR @{user}',  tweet_mode = "extended", count = n_tweets_user,
                        result_type = 'recent', since_id = _since_id, max_id = _max_id)

    #tweets #tweets arriba es una lista de statuses
    #dir(tweets[0]) # cada status tiene los siguentes metodos y atributos, el mas importante es ._json que convierte el status en un dicconario.
    n_queries_user +=1 
    n_queries += 1
        
    most_recent_id = tweets_batch.since_id
    num_none_rows = 0
    tweet_max_id = None
    
    
    while ((n_queries_user < max_queries_per_user) and (n_queries < max_queries)):
            if tweet_batch.max_id is not None:
                tweet_max_id = tweet_batch.max_id
            
            tweet_batch = api.search(q = f'to:{user} OR @{user}', lang = lang,
                                    count = n_tweets_user,
                                    max_id = tweet_max_id, 
                                    tweet_mode = 'extended',
                                    since_id = _since_id)
            n_queries_user += 1
            n_queries += 1
            tweets.extend(tweet_batch)
            if len(tweet_batch) == 0:
                num_none_rows += 1
            else:
                num_none_rows = 0

            # end while if max_id is lower than since_id 
            if (tweet_max_id is not None) and (_since_id is not None):
                if tweet_max_id < _since_id:
                    print('exited because tweet_max_id < since_id')
                    break

            if num_none_rows > 6: #if 6 searches in a row are none then stop searching for that user
                print('exited because there were 6 consecutive calls giving none')
                break
    
    tweets_mentions = []
    retweets_mentions = []
    newest_tweet_id_mentions = {}

    for idx, tweet in enumerate(tweets):
        
        access_time = datetime.now().strftime("%Y %b %d %H:%M:%S")
        date = datetime.strptime(tweet._json["created_at"], '%a %b %d %H:%M:%S %z %Y')
        date_5 = date - timedelta(hours = 5)            
        date_format = date_5.strftime("%Y %b %d %H:%M:%S") 

        # getting tweets
        if ('RT @' not in tweet._json["full_text"]):
            
            temp = [tweet._json["id"], access_time, date_format, tweet._json["user"]["screen_name"], 
                    None, False, None, tweet._json["full_text"]]
            tweets_mentions.append(temp)

        # getting retweets
        else:

            temp = [tweet._json["id"], access_time, date_format, 
                    tweet._json["user"]["screen_name"], None, True,
                    tweet._json["retweeted_status"]["user"]["screen_name"], 
                    tweet._json["retweeted_status"]["full_text"]]
            #print(temp)
            #break
            retweets_mentions.append(temp)
            
    if user not in newest_tweet_id_mentions:
        newest_tweet_id_mentions[user] = most_recent_id
        
    with open('newest_id_mentions.json', 'w') as outfile:
        json.dump(newest_tweet_id_mentions, outfile)
        
           
    print("Mentions Tweets and retweets collected")
    return tweets_mentions, retweets_mentions
    

tw_m, rtw_m = get_tweets_mentions(user)

NameError: name 'tweets_batch' is not defined

In [66]:
df_tw_m = pd.DataFrame(tw_m, columns=['id', 'fecha_consulta', 'fecha_escritura', 'cuenta_origen', 'query_busqueda',
                                   'retweet', 'in_replay_to', 'texto'])

df_rtw_m = pd.DataFrame(rtw_m, columns=['id', 'fecha_consulta', 'fecha_escritura', 'cuenta_origen', 'query_busqueda',
                                   'retweet', 'in_replay_to', 'texto'])


dfs2 = [df_tw_m, df_rtw_m]

t_rt_df_m = pd.concat(dfs2, ignore_index=True)

print(t_rt_df_m.head(20))


                     id        fecha_consulta       fecha_escritura  \
0   1357862360875204608  2021 Feb 05 20:34:04  2021 Feb 06 01:23:09   
1   1357859450837753860  2021 Feb 05 20:34:04  2021 Feb 06 01:11:35   
2   1357857969505984515  2021 Feb 05 20:34:04  2021 Feb 06 01:05:42   
3   1357857455930290177  2021 Feb 05 20:34:04  2021 Feb 06 01:03:39   
4   1357856815132905472  2021 Feb 05 20:34:04  2021 Feb 06 01:01:07   
5   1357853147486695425  2021 Feb 05 20:34:04  2021 Feb 06 00:46:32   
6   1357853132580155396  2021 Feb 05 20:34:04  2021 Feb 06 00:46:29   
7   1357852447092465664  2021 Feb 05 20:34:04  2021 Feb 06 00:43:45   
8   1357852083148500993  2021 Feb 05 20:34:04  2021 Feb 06 00:42:18   
9   1357851883940040705  2021 Feb 05 20:34:04  2021 Feb 06 00:41:31   
10  1357849948822380547  2021 Feb 05 20:34:04  2021 Feb 06 00:33:50   
11  1357849601882148865  2021 Feb 05 20:34:04  2021 Feb 06 00:32:27   
12  1357849049811083265  2021 Feb 05 20:34:04  2021 Feb 06 00:30:15   
13  13

In [67]:
t_rt_df_m.to_csv('/home/noone/Desktop/vfn/violencia_frontera_norte/tw_rtw_m.csv', sep = '\t', index = True, header=True)

In [68]:
var = 1357840643586940000

tweets = api.search(q = f'to:{user} OR @{user}',  tweet_mode = "extended", count = 100, result_type = 'recent', max_id = var)


tw_m, rtw_m = get_tweets_mentions(user)


df_tw_m = pd.DataFrame(tw_m, columns=['id', 'fecha_consulta', 'fecha_escritura', 'cuenta_origen', 'query_busqueda',
                                   'retweet', 'in_replay_to', 'texto'])

df_rtw_m = pd.DataFrame(rtw_m, columns=['id', 'fecha_consulta', 'fecha_escritura', 'cuenta_origen', 'query_busqueda',
                                   'retweet', 'in_replay_to', 'texto'])


dfs2 = [df_tw_m, df_rtw_m]

t_rt_df_m = pd.concat(dfs2, ignore_index=True)

print(t_rt_df_m.head(20))

t_rt_df_m.to_csv('/home/noone/Desktop/vfn/violencia_frontera_norte/tw_rtw_m2.csv', sep = '\t', index = True, header=True)

Mentions Tweets and retweets collected
                     id        fecha_consulta       fecha_escritura  \
0   1357840496522051587  2021 Feb 05 20:42:49  2021 Feb 05 23:56:16   
1   1357840003955580929  2021 Feb 05 20:42:49  2021 Feb 05 23:54:19   
2   1357838902397140993  2021 Feb 05 20:42:49  2021 Feb 05 23:49:56   
3   1357838097132716034  2021 Feb 05 20:42:49  2021 Feb 05 23:46:44   
4   1357836585262325764  2021 Feb 05 20:42:49  2021 Feb 05 23:40:43   
5   1357833777515528198  2021 Feb 05 20:42:49  2021 Feb 05 23:29:34   
6   1357831743055794178  2021 Feb 05 20:42:49  2021 Feb 05 23:21:29   
7   1357827704368857088  2021 Feb 05 20:42:49  2021 Feb 05 23:05:26   
8   1357825034040061957  2021 Feb 05 20:42:49  2021 Feb 05 22:54:49   
9   1357824854179909632  2021 Feb 05 20:42:49  2021 Feb 05 22:54:07   
10  1357824228498808832  2021 Feb 05 20:42:49  2021 Feb 05 22:51:37   
11  1357824150434426881  2021 Feb 05 20:42:49  2021 Feb 05 22:51:19   
12  1357824061649338371  2021 Feb 05 2

## Get user timeline tweepy

In [85]:
import tweepy
from twarc import Twarc
import pandas as pd
import json
import math
import datetime
from datetime import datetime, timedelta
import os

with open('creds.json') as json_file:
    creds = json.load(json_file)

In [192]:
  
def get_tweets_mentions3(user, petitions = 4, newest_id_possible_path='newest_id_mentions.json'):
    
    if os.path.exists(newest_id_possible_path):
        with open(newest_id_possible_path) as json_file:
            data = json.load(json_file)
            if user in data:
                _since_id = data[user]
            else:
                _since_id = None   
    else:
        _since_id = None
    
    _max_id = None
    
    
    print('_since_id', _since_id)
    print('_max_id', _max_id)


    n_queries = 0
    n_queries_user = 0
    max_queries = petitions
    
    
    auth = tweepy.AppAuthHandler(creds["client_key"], creds["client_secret"])
    api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

    tweets = tweet_batch = api.search(q = f'to:{user} OR @{user}',  tweet_mode = "extended", count = 100,
                        result_type = 'recent', since_id = _since_id, max_id = _max_id)

    #tweets #tweets arriba es una lista de statuses
    #dir(tweets[0]) # cada status tiene los siguentes metodos y atributos, el mas importante es ._json que convierte el status en un dicconario.
    n_queries += 1
        
    most_recent_id = tweet_batch.since_id
    num_none_rows = 0
    tweet_max_id = None
    
    
    while (n_queries < max_queries):
            if tweet_batch.max_id is not None:
                tweet_max_id = tweet_batch.max_id
            
            tweet_batch = api.search(q = f'to:{user} OR @{user}',
                                     result_type = 'recent',
                                    count = 100,
                                    max_id = tweet_max_id, 
                                    tweet_mode = 'extended',
                                    since_id = _since_id)
            n_queries += 1
            tweets.extend(tweet_batch)
            if len(tweet_batch) == 0:
                num_none_rows += 1
            else:
                num_none_rows = 0

            # end while if max_id is lower than since_id 
            if (tweet_max_id is not None) and (_since_id is not None):
                if tweet_max_id < _since_id:
                    print('exited because tweet_max_id < since_id')
                    break

            if num_none_rows > 6: #if 6 searches in a row are none then stop searching for that user
                print('exited because there were 6 consecutive calls giving none')
                break
    
    tweets_mentions = []
    retweets_mentions = []
    newest_tweet_id_mentions = {}

    for idx, tweet in enumerate(tweets):
        
        access_time = datetime.now().strftime("%Y %b %d %H:%M:%S")
        date = datetime.strptime(tweet._json["created_at"], '%a %b %d %H:%M:%S %z %Y')
        date_5 = date - timedelta(hours = 5)            
        date_format = date_5.strftime("%Y %b %d %H:%M:%S") 

        # getting tweets
        if ('RT @' not in tweet._json["full_text"]):
            
            temp = [tweet._json["id"], access_time, date_format, tweet._json["user"]["screen_name"], 
                    None, False, None, tweet._json["full_text"]]
            tweets_mentions.append(temp)

        # getting retweets
        else:

            temp = [tweet._json["id"], access_time, date_format, 
                    tweet._json["user"]["screen_name"], None, True,
                    tweet._json["retweeted_status"]["user"]["screen_name"], 
                    tweet._json["retweeted_status"]["full_text"]]
            #print(temp)
            #break
            retweets_mentions.append(temp)
            
    if user not in newest_tweet_id_mentions:
        newest_tweet_id_mentions[user] = most_recent_id
    
    if most_recent_id is not None:
        with open(newest_id_possible_path, 'w') as outfile:
            json.dump(newest_tweet_id_mentions, outfile)
        
           
    print("Mentions Tweets and retweets collected")
    return tweets_mentions, retweets_mentions

In [193]:
tw, rtw = get_tweets_mentions3(user)

_since_id 1358102498096140288
_max_id None
Mentions Tweets and retweets collected


In [194]:
len(tw)

0

In [195]:
len(rtw)

1